In [ ]:
import os
import pandas as pd
import janitor
import numpy as np
import forestplot as fp
import warnings

warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
plt.rcParams['font.monospace'] = ['Noto Mono', 'Tlwg Mono', 'Courier New']
from typing import Iterable, Optional

# ============================================================================
fp_loq_lod_values = "./data/edc-lod-loq.csv"
fp_cordblood_tf_estimates = "./data/pfas-tf-model5-cordblood.dta"
fp_cordblood_completeness = "./data/edc-gusto-completeness-tabulation.csv"

os.path.exists(fp_loq_lod_values)
os.path.exists(fp_cordblood_tf_estimates)
os.path.exists(fp_cordblood_completeness)

pfas_list = ["pfbs", "pfna", "pfoa", "pfos", "pfhxs", "pfba", "pfda", "pfunda"]

In [ ]:
try:
    df_cb = (
        pd.read_stata(fp_cordblood_tf_estimates)
        # ========================================================================
        # Convert the specs into a single column for the pfas name
        .fillna(0)
        .assign(pfas=lambda df_: pd.from_dummies(df_[pfas_list]))
        .remove_columns(pfas_list)
        # ========================================================================
        # Get LOD/LOQ values
        .merge(
            (
                pd.read_csv(fp_loq_lod_values).assign(
                    # LOD/LOQ have been divided by sqrt(2) for imputation
                    # Multiply by sqrt(2) to convert back to original
                    lod=lambda df_: df_["lod"] * np.sqrt(2),
                    loq=lambda df_: df_["loq"] * np.sqrt(2),
                    pfas=lambda df_: df_["chemical_name"].str.lower(),
                )
            ),
            how="left",
            on="pfas",
            validate="1:1",
        )
        # ========================================================================
        # Get completeness
        .merge(
            (pd.read_csv(fp_cordblood_completeness)),
            how="left",
            on="chemical_name",
            validate="1:1",
        )
        # ========================================================================
        # Format cell values for alignment
        # Pad the measured column
        .assign(measured=lambda df_: df_["measured"].apply(str).str.pad(len("   n")))
        # Pad the measured, < LOD, < LOQ columns
        # So that they are padded to the left
        .assign(
            **{
                k: lambda df_, col=k: df_[col]
                .apply(str)
                .str.pad(len(" < loq"), side="left")
                for k in ["LOD", "LOQ"]
            }
        )
        # Round the LOD/LOQ values and turn into string
        .assign(
            **{
                k: lambda df_, col=k: df_[col].apply(lambda x: "{:.{}f}".format(x, 3))
                for k in ["lod", "loq"]
            }
        )
        .assign(**{k: lambda df_, col=k: df_[col] + " ng/mL" for k in ["lod", "loq"]})
        .sort_values(
            ["measured", "loq", "lod"],
            ascending=[False, False, False],
            ignore_index=True,
        )
    )
    df_cb.to_csv(
        "./data/coefplot-pfas-transport-facilities-model5-cordblood.csv", index=False
    )
except:
    df_cb = (
        pd.read_csv("./data/coefplot-pfas-transport-facilities-model5-cordblood.csv")
        # ========================================================================
        # Format cell values for alignment
        # Pad the measured column
        .assign(measured=lambda df_: df_["measured"].apply(str).str.pad(len("   n")))
        # Pad the measured, < LOD, < LOQ columns
        # So that they are padded to the left
        .assign(
            **{
                k: lambda df_, col=k: df_[col]
                .apply(str)
                .str.pad(len(" < loq"), side="left")
                for k in ["LOD", "LOQ"]
            }
        ).sort_values(
            ["measured", "loq", "lod"],
            ascending=[False, False, False],
            ignore_index=True,
        )
    )

df_cb

In [ ]:
def save_mpl_fig(
    savepath: str, formats: Optional[Iterable[str]] = None, dpi: Optional[int] = None
) -> None:
    """Save matplotlib figures to ../output.

    Will handle saving in png and in pdf automatically using the same file stem.

    Parameters
    ----------
    savepath: str
        Name of file to save to. No extensions.
    formats: Array-like
        List containing formats to save in. (By default 'png' and 'pdf' are saved).
        Do a:
            plt.gcf().canvas.get_supported_filetypes()
        or:
            plt.gcf().canvas.get_supported_filetypes_grouped()
        To see the Matplotlib-supported file formats to save in.
        (Source: https://stackoverflow.com/a/15007393)
    dpi: int
        DPI for saving in png.

    Returns
    -------
    None
    """
    # Save pdf
    plt.savefig(f"{savepath}.pdf", dpi=None, bbox_inches="tight", pad_inches=0)

    # save png
    plt.savefig(f"{savepath}.png", dpi=dpi, bbox_inches="tight", pad_inches=0)

    # Save additional file formats, if specified
    if formats:
        for format in formats:
            plt.savefig(
                f"{savepath}.{format}",
                dpi=None,
                bbox_inches="tight",
                pad_inches=0,
            )
    return None

In [ ]:
selected_pfas_opts = dict(
    estimate="beta",
    ll="l95",
    hl="u95",
    varlabel="chemical_name",
    pval="pval",
    annote=["measured", "LOD", "LOQ", "lod", "loq"],
    annoteheaders=["   N", " < LOD", " < LOQ", "  LOD value", "  LOQ value"],
    rightannote=["est_ci", "formatted_pval"],
    right_annoteheaders=["Est. (95% Conf. Int.)", "P-value"],
    table=True,
    variable_header="PFAS",
    decimal_precision=3,
    xlabel="Standardized estimate",
    xlabel_size=17,
    xticks=[-0.4, -0.2, 0, 0.2, 0.4],
    symbols=["$^a$", "$^b$", "$^c$"],
    **{
        "marker": "o",  # set maker symbol as diamond
        "markersize": 120,  # adjust marker size
        "xlinestyle": (0, (10, 5)),  # long dash for x-reference line
        "xlinecolor": "#808080",  # gray color for x-reference line
        "xtick_size": 18,  # adjust x-ticker fontsize
        "fontsize": 20,
    },
)

fp.forestplot(
    df_cb.query("pfas in @pfas_list"),
    **selected_pfas_opts,
)
save_mpl_fig("./figures/coefplot-8pfas-transport-facilities-model5-cordblood")